# Eliminación de preguntas que no cumplen los criterios
Con el dataset limpio, podemos comenzar a realizar un análisis utilizando algunas técnicas de NLP para eliminar aquellas preguntas que no cumplan con los criterios del INE establecidos en el punto

El resultado de este paso serán:
- Un archivo que contenga el texto de las preguntas que no cumplen los criterios y una breve explicación de porqué no está cumpliéndolos (discurso de odio, violencia, malas palabras, etc.)

- Un archivo con las preguntas restantes (los campos serán los mismos que en el punto 3.1)

- ID del registro

- Entidad de Origen

- Edad

- Género

- Identificación con algún grupo en situación de discriminación

- Tema de la pregunta

- Texto de la pregunta

In [1]:
!pip install pysentimiento

  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylint 2.7.4 requires astroid<2.7,>=2.5.2, but you have astroid 2.5 which is incompatible.



  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.9.0
    Uninstalling fsspec-0.9.0:
      Successfully uninstalled fsspec-0.9.0


In [9]:
import pandas as pd

from pysentimiento import create_analyzer

hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

limpio_df = pd.read_csv('./Base_limpia.csv')

for pregunta in limpio_df['pregunta']:
    analizador = hate_speech_analyzer.predict(pregunta).output
    if len(analizador) > 0:
      print(pregunta)
      with open('./preguntas_rechazadas.txt', 'a') as archivo:
        archivo.write(f'La pregunta "{pregunta}" fue rechazada por: {analizador}')
      limpio_df = limpio_df[limpio_df['pregunta'] != pregunta]


display(limpio_df)

¿Cuando van a dejar de permitir que los extranjeros y migrantes ataquen a los mexicanos en su propia casa?


KeyError: "['¿Cuando van a dejar de permitir que los extranjeros y migrantes ataquen a los mexicanos en su propia casa?'] not found in axis"